In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from functools import reduce
import pandas as pd
from urllib.parse import urljoin

In [ ]:
#URL de la pagina con la cual trabajaremos
main_page = "https://books.toscrape.com"

response = requests.get(main_page)

if response.status_code == 200:
    content = response.text
    data = BeautifulSoup(content, "html.parser")

    #Definimos Constantes con informacion que sera usada en varios ejercicios
    
    LINES = f"{"-":->49}"
    #Buscamos etiqueta article con clase 'product_pod' (es la que contiene toda la info de los libros)
    DATA_BOOKS = data.select("article.product_pod")
    
    #Accedemos a la etiqueta div con clase 'product_price' y a su hijo 'p' con clase 'price_color' para de alli obtener el valor del texto
    #   eliminando caracteres innecesarios y almacenandolos como datos flotantes
    PRICES_VALUES = [float(price.get_text().replace("Â£", "")) for price in data.select("div.product_price > p.price_color")]   

else: 
    print(f"Error code: {response.status_code}") 

In [ ]:
print(f"{LINES}\nEjercicios 1.1-1.3:\n")

#1.1 Extraer el título del primer libro en la página principal y muéstralo en la consola.

#Buscamos etiquetas 'a' que contengan un href y este inicie con la palabra catalogue, ademas el atributo title debe tener caracteres 
a_tag = data.find(name="a", attrs={"href": re.compile(r"^catalogue/"), "title": re.compile(r"\w")}) 
print(f"First book title: {a_tag["title"]}")

#1.2 Encuentra el precio del primer libro en la página principal.

#Buscamos un div que tenga la clase 'product_price' y que este tenga una etiqueta hijo 'p' con la clase 'price_color'
first_price = data.select_one("div.product_price > p.price_color")
#Obtenemos texto de etiqueta y eliminamos caracter innecesario
first_price = first_price.get_text().replace("Â","")
print(f"First book price: {first_price}")

#1.3 Obtén la calificación (como texto o número) del primer libro en la página principal.

#Buscamos un article con la clase 'product_pod' y accedemos a su hijo 'p', del cual accederemos a su attr clase que contiene la cantidad de estrellas
calification = data.select_one("article.product_pod > p")
print(f"First book calification: {calification["class"][1]}")

print(f"{LINES}\nEjercicio 1.4:\n")

#1.4 Navega a la barra lateral y extrae todas las categorías de libros disponibles, mostrando el nombre de cada categoría.

#Accedemos a la ul dentro del div con clase 'side_categories' y recorremos cada categoria eliminando separadores y mostrando en pantalla
categories = data.select("div.side_categories > ul.nav-list > li > ul")
print("Categories: ", sep=" ")
for category in categories:
    print(f"{category.get_text(separator="\n", strip=True)}")

print(f"{LINES}\nEjercicio 1.5:\n")

#1.5 Extraer los titulos, calificaciones y precios de los libros en la página principal y muéstralos en la consola con un contador.

#Definimos diccionario con numero de estrellas y una representacion mas agradable de las mismas
starts = {"zero" : "Ǿ", "one" : "♥", "two" : "♥♥", "three" : "♥♥♥", "four" : "♥♥♥♥", "five" : "♥♥♥♥♥"}
#Obtenemos titulos y precios
titles = data.findAll(name="a", attrs={"href": re.compile(r"^catalogue/"), "title": re.compile(r"\w")})
#Usamos list comprehension para almacenar los precios sin el caracter no deseado
califications = data.select("article.product_pod > p")

#Recorremos cada una de las anteriores listas e inicializamos contador del libro en 0
num_book = 0
for title, price, calification in zip(titles, PRICES_VALUES, califications):
    #Accedemos al segundo valor del atributo clase en el cual se encuentra el numero de estrellas en forma de texto
    calification = calification["class"][1].lower()
    #Redefinimos ese valor por su correspondiente representacion en el diccionario y mostramos la info en pantalla
    calification =  starts[calification]
    num_book += 1
    print(f"Book #{num_book}: {title["title"]} (£{price}) {calification}")
           
   

In [ ]:
print(f"{LINES}\nEjercicios 2.1-2.3:\n")

#2.1 Obtén el enlace de la imagen de la portada del primer libro y guárdalo en una variable.

#Buscamos etiqueta div con la clase image_container y accedemos a su hijo a, del cual extramos el valor del atributo 'href'
first_link = data.select_one("div.image_container > a")
first_link = first_link["href"]
print(f"Link first book: {first_link}")

#2.2 Cuenta el número de libros que aparecen en la primera página de la tienda y muestra el total.

#Buscamos etiqueta li con la clase correspondiente a los libros y recorremos cada elemento contando cada iteracion (esto representa la cantidad de libros)
books = data.find_all(name="li", attrs={"class" : "col-xs-6 col-sm-4 col-md-3 col-lg-3"})
num_book = 0
for book in books:
    num_book += 1
    
print(f"There are {num_book} books in the main page")

#2.3 Extrae el enlace a la siguiente página de libros, si existe, y guárdalo en una variable.

#Buscamos etiqueta li con clase 'next' y accedemos a su hijo 'a' extrayendo su link (valor href)
next_link = data.select_one("li.next > a")
next_link = next_link["href"]
print(f"The link of the next page is: {next_link}")

#2.4 Extrae los títulos de todos los libros en la página principal que tengan una calificación de cuatro o cinco estrellas.

print(f"{LINES}\nEjercicio 2.4:")
print("\nBooks with 4 or 5 starts: ")
#Recorremos cada libro y accedemos a la etiqueta 'p', extrayendo de ella el segundo valor del atributo clase (el cual contiene calificacion en texto)
for book in DATA_BOOKS:
    start = book.p["class"][1].lower()
    #Filtramos libros con 4 o 5 estrellas y mostramos info
    if start == "four" or start == "five":
        title = book.h3.a["title"]
        print(f"Title: {title}, starts: {start}")

#2.5 Encuentra el libro con el precio más alto en la primera página y muestra el título y el precio.

print(f"{LINES}\nEjercicio 2.5:")

#Recorremos la constante con la info de cada libro y accedemos a su hijo 'h3' y al nieto 'a' del cual extraemos el valor del atributo 'title'
titles = [book.h3.a['title'] for book in DATA_BOOKS]
#Usamos dict comprenhension para almacenar titulo y precio de cada libro
title_price = {title : price for title, price in zip(titles, PRICES_VALUES)}
max_title = ""
max_price = 0

#Recorremos cada precio y si es el maximo y almacenamos el maximo en la correspondiente variable con su nombre
for title, price in title_price.items():
    if price > max_price:
        max_price = price
        max_title = title

print(f"\nEl libro con el precio mas alto es: {max_title}, y su valor es: £{max_price}")

In [ ]:
print(f"{LINES}\nEjercicio 3.1:\n")

#3.1 Calcula el precio promedio de todos los libros en la página principal. 

#De la constante con los precios usamos reduce para obtener la suma de todos 
prices_sum = reduce(lambda x, y: x + y, PRICES_VALUES)
#Redondeamos el resultado de la suma a 2 valores y lo dividimos entre la cantidad de libros
print(f"Main page's average: £{round(prices_sum/len(PRICES_VALUES), 2)}\n")

print(f"{LINES}\nEjercicio 3.2:\n")

#3.2 Filtra los libros en la página principal con calificaciones de tres estrellas o menos. Guarda el título, el precio y la calificación en un DF y archivo CSV.

filter_books = []
#Recorremos las constantes con la data y los precios de los libros en la pagina principal y accedemos a sus valores del titulo y calificacion
for book, price in zip(DATA_BOOKS, PRICES_VALUES):
    title = book.h3.a["title"]
    star = book.p["class"][1].lower()
    #Filtramos libros con 1,2 o 3 estrellas y los agregamos a la lista ('filter_books') en forma de lista (lista con los datos de cada iteracion)
    if star in ("one", "two", "three"):
        filter_books.append([title, f"£{price}", star])
        
#Convertimos la lista en DF de pandas almacenando los valores para despues crear el archivo csv con la data
filter_books = pd.DataFrame(filter_books, columns=["Title", "Price", "Calification"])
filter_books.to_csv("lessThan3Stars.csv", sep=";")
print(f"{filter_books}\n")

print(f"{LINES}\nEjercicio 3.3:\n")

#3.3 Encuentra todos los libros con cinco estrellas en la página principal y descarga sus imágenes de portada en una carpeta local. Nombrar cada archivo de 
# imagen usando el título del libro y el formato .jpg.

print(f"Book with five stars:") 
for book in DATA_BOOKS:
    #Recorremos contenedor general de cada libro para obtener la calificacion en forma de texto y filtrar los que cuenten con 5 estrellas 
    stars = book.p["class"][1].lower()
    if stars == "five":
        #De los libros filtrados obtenemos sus titulos y link de las imagenes
        title = book.div.a.img["alt"]
        img_link = book.div.a.img["src"]
        #Hacemos la peticion del recurso de la imagen 
        img = requests.get(f"{main_page}/{img_link}")
        print(f"\t{title}")
        #Creamos el archivo binario con la imagen y lo almacenamos en la carpeta files
        with open (fr"files\{title}.jpg", "wb") as image:
            image.write(img.content)
            
            
print(f"{LINES}\nEjercicio 3.4:\n")

#3.4 Extraer categoría y cantidad de libros de cada categoría en la página principal. Mostrar el nombre de la categoría junto con el total de libros que tiene.

categories = []
for book in DATA_BOOKS:
    #Recorremos cada libro y obtenemos toda su URL para hacer esta solicitud al servidor 
    book_url = main_page + f"/{book.h3.a['href']}"
    
    response = requests.get(book_url)
    if response.status_code == 200:
        content = response.text
        book_data = BeautifulSoup(content, "html.parser")
        #Accedemos al header en el cual se anexa la categoria a la que pertenece y obtenemos el nombre, para finalmente agregar la categoria a la lista de categorias
        header = book_data.select("ul.breadcrumb > li")
        category = header[2].get_text()
        categories.append(category)
        
    else: 
        print(f"Error code: {response.status_code}")    
    
#Borramos saltos de linea 
categories = list(map(lambda x: x.strip("\n"), categories))

#Contamos cada aparicion de la categoria y almacenamos en un diccionario {category_name : conteo} (Si la categoria ya fue contada se redefine, garantizando que no se repita)
categories_count = {}
for category in categories:
    categories_count[category] = categories.count(category) 
    
#Recorremos e imprimos las key y values del diccionario con el conteo 
for k, v in categories_count.items():
    print(f"{k}: {v}") 
    
    
print(f"{LINES}\nEjercicio 3.5:\n")
    
#3.5 Navegar a cada categoría (puedes seguir los enlaces de la barra lateral) y extraer el título y el precio del libro más barato y más caro en cada categoría. 

"""En ocasiones al hacer tantas solicitudes al servidor, este se puede bloquear y dar un error 404 en la busqueda de 
los libros maximos y minimos (Espera un momento para ejecutar el ejercicio 3.5)"""

#Obtenemos el link de cada categoria en el sidebar
cat_links = data.select("div.side_categories > ul.nav-list > li > ul")
for link in cat_links:
    #Recorremos cada link y accedemos a su etiqueta 'li' la cual estan los nombres de la categoria y su link (almacenamos nombre, link en forma de tupla dentro de la lista para cada categoria) 
    cat_data = [(item.a.get_text().strip(), item.a["href"]) for item in link.find_all("li")]  
    
for category in cat_data:
    max_name = ""
    max_price = 0
    min_name = ""
    min_price = 1000000
    name = category[0]
    #Obtenemos la ruta de cada categoria y la almacenamos en una variable
    cat_page = urljoin(main_page, category[1])
    print(f"Category: {name}")
    
    #Hacemos el get de cada pagina de categoria
    response = requests.get(cat_page)
    if response.status_code == 200:
        content = response.text
        data = BeautifulSoup(content, "html.parser")
        
        #Obtenemos titulos y precios de cada libro de la categoria de la iteracion
        books = data.select("article.product_pod")
        titles = [book.h3.a.get_text() for book in books]
        prices = [float(book.div.find_next_sibling("div").p.get_text().replace("Â£", "")) for book in books]
        dict_data = {title: price for title, price in zip(titles, prices)}
        
        #Comparamos el precio y en caso de ser mayor o menor se almacena en la correspondiente varible, cuando son iguales se concantena los nombres
        for title, price in dict_data.items():
            if price > max_price:
                max_price = price
                max_name = title
            elif price == max_price:
                max_price = price
                max_name += f", {title}" 
            
            if price < min_price:
                min_price = price
                min_name = title
            elif price == min_price:
                min_price = price
                min_name += f", {title}" 
            
        print(f"\tMaximum: {max_name}, Price: £{max_price}")
        print(f"\tMinimum: {min_name}, Price: £{min_price}")
        
    else: 
        print(f"Error code: {response.status_code}")    

    print(f"{"-":->19}")  